In [1]:
!pip install lyricsgenius

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
dir(genius)

NameError: name 'genius' is not defined

In [1]:
import lyricsgenius
genius = lyricsgenius.Genius("dU0w3p_pSek4AvwkaiqKGwl05lMqIl1jVWhax3S66imLphoiO4UqLDJjsZwyXzZy", verbose=False)

In [3]:
df = pd.read_csv("labeled_data.csv")
songs = []
for index, song in df.iterrows()[:5]:
    title = song['song']
    artist = song['artist']
    songs.append(genius.search_song(artist + " " + title, get_full_info=False))

NameError: name 'read_csv' is not defined

In [27]:
import re

def clean_song(song):
    return '\n'.join(line.strip() for line in re.sub(' +', ' ', re.sub('[^a-zA-Z0-9 \',?!;\n]', ' ', song.lyrics)).splitlines())

In [28]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [215]:
seeds = set(['song'])
extraction_patterns_to_avoid = set(['CCONJ', 'SPACE', 'PUNCT', 'PART'])
extraction_patterns = []
extraction_patterns_for_line = []

# songs = [yellow, mockingbird, high_on_life, church]
songs = [yellow]

In [216]:
for song in songs:
    analyzed_song = nlp(clean_song(song))
    for sent in analyzed_song.sents:
        for token in sent:
            if token.text in seeds or token.lemma_ in seeds:
                pattern = []
                for _token in sent:
                    if _token.pos_ not in extraction_patterns_to_avoid:
                        if _token.text in seeds or _token.lemma_ in seeds:
                            pattern.append(_token.pos_)
                        else:
                            pattern.append(_token.pos_)
                extraction_patterns.append(pattern)
                break

In [217]:
for song in songs:
    for line in clean_song(song).splitlines():
        analyzed_line = nlp(line)
        for sent in analyzed_line.sents:
            for token in sent:
                if token.text in seeds or token.lemma_ in seeds:
                    pattern = []
                    for _token in sent:
                        if _token.pos_ not in extraction_patterns_to_avoid:
                            if _token.text in seeds or _token.lemma_ in seeds:
                                pattern.append(_token.pos_)
                            else:
                                pattern.append(_token.pos_)
                    extraction_patterns_for_line.append(pattern)
                    break

In [219]:
extraction_patterns, extraction_patterns_for_line

([['PRON',
   'VERB',
   'DET',
   'NOUN',
   'ADP',
   'PRON',
   'DET',
   'DET',
   'NOUN',
   'PRON',
   'VERB']],
 [['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON']])

In [220]:
for song in songs:
    analyzed_song = nlp(clean_song(song))
    for sent in analyzed_song.sents:
        pattern = []
        tokens = []
        for token in sent:
            if token.pos_ not in extraction_patterns_to_avoid:
                pattern.append(token.pos_)
                tokens.append(token.text)
        if pattern in extraction_patterns:
#             print(sent)
            print(pattern)
            print(tokens)
            print()

['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON', 'DET', 'DET', 'NOUN', 'PRON', 'VERB']
['I', 'wrote', 'a', 'song', 'for', 'you', 'all', 'the', 'things', 'you', 'do']



In [221]:
for song in songs:
    for line in clean_song(song).splitlines():
        analyzed_line = nlp(line)
        for sent in analyzed_line.sents:
            pattern = []
            for token in sent:
                pattern.append(token.pos_)
            if pattern in extraction_patterns_for_line:
                print(sent)
                print(pattern)

I wrote a song for you
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON']
I drew a line for you
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON']


In [4]:
query = "eminem love the way you lie"
ACCESS_TOKEN = "dU0w3p_pSek4AvwkaiqKGwl05lMqIl1jVWhax3S66imLphoiO4UqLDJjsZwyXzZy"

In [5]:
import requests
import json

In [6]:
def search(query):
    response = requests.get("http://api.genius.com/search?q={}&access_token={}".format(query, ACCESS_TOKEN))
    response_json = json.loads(response.content)
    underscored_line = ''.join(["_" for _ in range(117)])
    first = None
    try:
        first = response_json['response']['hits'][0]
    except e as Exception:
        first = None
    if first is not None:
        if first['type'] == 'song':
            song_output = {}
            song_text = ""
            song_id = first['result']['id']
            title = first['result']['title']
            artist = first['result']['primary_artist']['name']
            annotation_count = first['result']['annotation_count']
            
            song_response = requests.get("http://api.genius.com/songs/{}?access_token={}&text_format=plain".format(song_id, ACCESS_TOKEN))
            song_response_json = json.loads(song_response.content)
            song_description = song_response_json['response']['song']['description']['plain']
            
            print("Found: {} by {} with {} annotations".format(title, artist, annotation_count))
            print("\nSong description:\n{}\n{}\n{}".format(underscored_line,
                                                           song_description,
                                                           underscored_line
                                                      ))
            song += song_
            referent_response = requests.get("http://api.genius.com/referents?song_id={}&access_token={}&text_format=plain".format(song_id, ACCESS_TOKEN))
            referent_response_json = json.loads(referent_response.content)
            for referent in referent_response_json['response']['referents']:
                print("Fragment:\n{}\n{}\n{}".format(underscored_line, referent['fragment'], underscored_line))
                i = 1
                print(underscored_line)
                for annotation in referent['annotations']:
                    if 'plain' in annotation['body']:
                        print("{}.".format(i),annotation['body']['plain'])
                        i += 1
                print(underscored_line)
            
            return song_response_json, referent_response_json
        else:
            print("No song found")
        print(first['result'])
    else:
        print("Found nothing")

In [7]:
song, ref = search("Michael Jackson Dangerous")

Found: Dangerous by Michael Jackson with 10 annotations

Song description:
_____________________________________________________________________________________________________________________
“Dangerous” is the titular closing track to Jackson’s 1991 album. Despite its upbeat tempo and addictive rhythm, it was not released as a single.

The song has similar themes to Jackson’s 1987 song, “Dirty Diana”: it talks about a seductress whom Jackson falls for, even though he knows she is no good for him.
_____________________________________________________________________________________________________________________
Fragment:
_____________________________________________________________________________________________________________________
And then it happened
 She touched me
 For the lips of
 A strange woman
 Drop as a honeycomb
 And her mouth was
 Smoother than oil
 But her inner spirit and words
 Were as sharp as
 A two-edged sword
 But I loved it
 Cause it’s dangerous
_____________

In [8]:
song

{'meta': {'status': 200},
 'response': {'song': {'annotation_count': 10,
   'api_path': '/songs/118470',
   'apple_music_id': '322847216',
   'apple_music_player_url': 'https://genius.com/songs/118470/apple_music_player',
   'description': {'plain': '“Dangerous” is the titular closing track to Jackson’s 1991 album. Despite its upbeat tempo and addictive rhythm, it was not released as a single.\n\nThe song has similar themes to Jackson’s 1987 song, “Dirty Diana”: it talks about a seductress whom Jackson falls for, even though he knows she is no good for him.'},
   'embed_content': "<div id='rg_embed_link_118470' class='rg_embed_link' data-song-id='118470'>Read <a href='https://genius.com/Michael-jackson-dangerous-lyrics'>“Dangerous” by Michael\xa0Jackson</a> on Genius</div> <script crossorigin src='//genius.com/songs/118470/embed.js'></script>",
   'featured_video': False,
   'full_title': 'Dangerous by\xa0Michael\xa0Jackson',
   'header_image_thumbnail_url': 'https://images.genius.com/

In [9]:
ref

{'meta': {'status': 200},
 'response': {'referents': [{'_type': 'referent',
    'annotator_id': 890652,
    'annotator_login': 'Tommoslav',
    'api_path': '/referents/3270292',
    'classification': 'accepted',
    'fragment': 'And then it happened\n She touched me\n For the lips of\n A strange woman\n Drop as a honeycomb\n And her mouth was\n Smoother than oil\n But her inner spirit and words\n Were as sharp as\n A two-edged sword\n But I loved it\n Cause it’s dangerous',
    'id': 3270292,
    'is_description': False,
    'path': '/3270292/Michael-jackson-dangerous/And-then-it-happened-she-touched-me-for-the-lips-of-a-strange-woman-drop-as-a-honeycomb-and-her-mouth-was-smoother-than-oil-but-her-inner-spirit-and-words-were-as-sharp-as-a-two-edged-sword-but-i-loved-it-cause-its-dangerous',
    'range': {'content': "And then it happened\n She touched me\n For the lips of\n A strange woman\n Drop as a honeycomb\n And her mouth was\n Smoother than oil\n But her inner spirit and words\n W